# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import cartopy
import geoviews
# Import API key
from api_keys import geoapify_key

c:\ProgramData\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
c:\ProgramData\anaconda3\envs\dev\lib\site-packages\geoviews\operation\__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.17, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,4.59,92,100,7.48,SJ,1691530563
1,1,howrah,22.5892,88.3103,27.00,94,75,2.06,IN,1691530594
2,2,tiksi,71.6872,128.8694,15.04,53,9,1.47,RU,1691530330
3,3,bethel,41.3712,-73.4140,26.20,63,40,5.14,US,1691530560
4,4,thompson,55.7435,-97.8558,14.09,82,100,4.63,CA,1691530355


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot("Lng","Lat",c='City', kind='points',s='Humidity', scale=0.5, geo=True,tiles=True)
# Display the map
# YOUR CODE HERE


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
vacation_df = city_data_df.loc[(city_data_df["Max Temp"]>= 25) & (city_data_df["Max Temp"]<30) &
                              (city_data_df["Cloudiness"]<25) & (city_data_df["Humidity"]<30)]
# Drop any rows with null values
# YOUR CODE HERE
vacation_df.dropna()
# Display sample data
# YOUR CODE HERE
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,andkhoy,36.9529,65.1238,27.46,27,0,3.57,AF,1691530888
30,30,serdar,38.9764,56.2757,29.77,25,0,2.47,TM,1691530892
64,64,ribas do rio pardo,-20.4431,-53.7592,28.35,28,3,2.45,BR,1691530898
77,77,kashgar,39.4547,75.9797,25.56,27,20,4.00,CN,1691530900
118,118,pervari,37.9357,42.5493,25.92,28,0,2.65,TR,1691530908


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = vacation_df[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df.insert(5,'Hotel Name', '')
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,andkhoy,AF,36.9529,65.1238,27,
30,serdar,TM,38.9764,56.2757,25,
64,ribas do rio pardo,BR,-20.4431,-53.7592,28,
77,kashgar,CN,39.4547,75.9797,27,
118,pervari,TR,37.9357,42.5493,28,


 Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    'apiKey': geoapify_key,
    'limit': 10,
    'categories': 'accommodation.hotel'
}
    # YOUR CODE HERE


# # Print a message to follow up the hotel search
print("Starting hotel search")

# # Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
# #     # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    # params["bias"] = f"bias=proximity:{long},{lat}" # YOUR CODE HERE
    
#     # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


#     # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params) # YOUR CODE HERE
#     # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
#     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    

# # Display sample data
hotel_df

Starting hotel search
andkhoy - nearest hotel: No hotel found
serdar - nearest hotel: Chynar Hotel
ribas do rio pardo - nearest hotel: Hotel Espírito Santo
kashgar - nearest hotel: Yambu Grand Hotel
pervari - nearest hotel: No hotel found
kandahar - nearest hotel: No hotel found
ambon - nearest hotel: Swiss-belhotel Ambon
al jawf - nearest hotel: No hotel found
ta'if - nearest hotel: فندق سفن جاردن
elko - nearest hotel: No hotel found
dalbandin - nearest hotel: SF Al-dawood Restaurant


,City,Country,Lat,Lng,Humidity,Hotel Name
6,andkhoy,AF,36.9529,65.1238,27,No hotel found
30,serdar,TM,38.9764,56.2757,25,Chynar Hotel
64,ribas do rio pardo,BR,-20.4431,-53.7592,28,Hotel Espírito Santo
77,kashgar,CN,39.4547,75.9797,27,Yambu Grand Hotel
118,pervari,TR,37.9357,42.5493,28,No hotel found
297,kandahar,AF,31.0000,65.7500,15,No hotel found
398,ambon,ID,-3.6954,128.1814,15,Swiss-belhotel Ambon
401,al jawf,SA,29.5000,38.7500,17,No hotel found
485,ta'if,SA,21.2703,40.4158,15,فندق سفن جاردن
548,elko,US,41.0002,-115.5012,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points('Lng', 'Lat', geo = True, tiles = 'OSM', frame_width = 700, frame_height = 500, scale = 1, color = 'City', fill_alpha = 1, hover_cols=['Hotel Name', 'Country'])
# Display the map
# YOUR CODE HERE
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)